In [3]:
import pandas as pd
import csv

listings = []
csv_arr = []
file_name = 'data/10-27-2017.tweets.xlsx'
xls = pd.ExcelFile(file_name)
print(xls.sheet_names)
for sheetname in [xls.sheet_names[0], xls.sheet_names[1]]:
    listing = pd.read_excel(file_name, sheet_name=sheetname)
    listings.append(listing)

for listing in listings:
    # print(list(listing))
    # get "T/F" location
    answer_index = (list(listing)).index("T/F")
    # get "Tweet Text" location
    text_index = (list(listing)).index("Tweet Text")
    for index, row in listing.iterrows():
        # skip 1st row as it is headers
        if index == 0:
            continue

        answer = row[answer_index]
        text = row[text_index]

        if answer == 0:
            answer = 'spam'
        else:
            answer = 'ham'
        csv_arr.append([answer, text])
print(len(csv_arr))

# write csv
with open('test.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',',
            quotechar='"', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(['v1', 'v2'])

    for row in csv_arr:
        spamwriter.writerow(row)

['10-27-2017.tweets', '10-27-2017.tweets.TRUE', 'RT']
181


In [4]:
from sklearn.naive_bayes import *
from sklearn.dummy import *
from sklearn.ensemble import *
from sklearn.neighbors import *
from sklearn.tree import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.calibration import *
from sklearn.linear_model import *
from sklearn.multiclass import *
from sklearn.svm import *
import pandas
import csv

data = pandas.read_csv('test.csv', sep=',', header=0)
#print(data.columns.tolist())

# we have total 182 rows in onebig.csv
# use 100 for training
train_data = data[:100] 
test_data = data[100:] 
print('train data:')
print((train_data.v1).value_counts())
print('test data:')
print(((test_data.v1).value_counts()))
classifier = OneVsRestClassifier(SVC(kernel='linear'))
vectorizer = TfidfVectorizer()

# train
vectorize_text = vectorizer.fit_transform(train_data.v2)
classifier.fit(vectorize_text, train_data.v1)

# score
vectorize_text = vectorizer.transform(test_data.v2)
score = classifier.score(vectorize_text, test_data.v1)
print('score: {}'.format(score))


csv_arr = []
for index, row in test_data.iterrows():
    answer = row[0]
    text = row[1]
    vectorize_text = vectorizer.transform([text])
    predict = classifier.predict(vectorize_text)[0]
    if predict == answer:
        result = 'right'
    else:
        result = 'wrong'
    csv_arr.append([len(csv_arr), text, answer, predict, result])


# write csv
with open('test_score.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=';',
            quotechar='"', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(['#', 'text', 'answer', 'predict', result])

    for row in csv_arr:
        spamwriter.writerow(row)

train data:
ham     55
spam    45
Name: v1, dtype: int64
test data:
ham     80
spam     1
Name: v1, dtype: int64
score: 0.9135802469135802
